In [63]:
from tqdm import tqdm
import json
import os
import pandas as pd

df_sample = pd.read_csv("./data/294-doctrines/legal_doctrines_294.tsv", delimiter="\t")

print(df_sample.shape)
df_sample.head(3)


(294, 3)


,concept,intro_text,word_count
0,Abatement_ab_initio,"Abatement ab initio (Latin for ""from the begin...",85
1,Abstention_doctrine,An abstention doctrine is any of several doctr...,159
2,Abstraction_principle_(law),The abstract system of title transfer (German:...,96


## 1. organize concept, definitions, and stories in `tsv` files

In [121]:
def organize_doctrine_stories(model_name):
    lines = []
    for concept, text in tqdm(zip(df_sample.concept.to_list(), df_sample.intro_text.to_list())):
        lines.append([concept, text])
        filename = os.path.join("./outputs/294-doctrines-{}/story/".format(model_name), "{}.json".format(concept))
        with open(filename) as f:
            jobj = json.load(f)
            lines[-1].append(jobj['annotation'].replace("Concept Simplified:", "").strip("\n"))

    df_annotations = pd.DataFrame(lines, columns=['concept', 'intro_text', 'story'])
    df_annotations.story = df_annotations.story.str.strip()
    return df_annotations


df_llama2_stories = organize_doctrine_stories("llama2")
df_llama2_stories.to_csv('./outputs/294-doctrines-llama2/294_doctrine_stories.tsv', sep="\t")
df_gpt35_stories = organize_doctrine_stories("gpt3.5")
df_gpt35_stories.to_csv('./outputs/294-doctrines-gpt3.5/294_doctrine_stories.tsv', sep="\t")
df_gpt4_stories = organize_doctrine_stories("gpt4")
df_gpt4_stories.to_csv('./outputs/294-doctrines-gpt4/294_doctrine_stories.tsv', sep="\t")


294it [00:00, 6772.66it/s]
294it [00:00, 7724.41it/s]
294it [00:00, 8256.61it/s]


In [122]:
assert sum(df_llama2_stories.story == df_gpt35_stories.story) == 0, "same story detected"
assert sum(df_llama2_stories.story == df_gpt4_stories.story) == 0, "same story detected"
assert sum(df_gpt35_stories.story == df_gpt4_stories.story) == 0, "same story detected"

assert sum(df_llama2_stories.intro_text == df_gpt35_stories.intro_text) == 294, "not all definitions matched"
assert sum(df_llama2_stories.intro_text == df_gpt4_stories.intro_text) == 294, "not all definitions matched"
assert sum(df_gpt35_stories.intro_text == df_gpt4_stories.intro_text) == 294, "not all definitions matched"

# 2. organize stories + questions in `tsv` format

In [123]:
def add_questions(df_story, folder_path):

    for question_type in ['concept_question', 'ending_question', 'limitation_question']:
        lines = []
        for concept in tqdm(df_story.concept.to_list()):
            filename = os.path.join("{}/{}/".format(folder_path, question_type), "{}.json".format(concept))
            with open(filename) as f:
                jobj = json.load(f)
                lines.append(jobj['annotation'])

        df_story[question_type] = lines
    return df_story

print("llama2:")
df_llama2_stories_questions = add_questions(df_llama2_stories, "./outputs/294-doctrines-llama2")
df_llama2_stories_questions.to_csv("./data/294-doctrines/llama2_story_question_294.tsv", sep="\t")

print("gpt3.5:")
df_gpt35_stories_questions = add_questions(df_gpt35_stories, "./outputs/294-doctrines-gpt3.5")
df_gpt35_stories_questions.to_csv("./data/294-doctrines/gpt3.5_story_question_294.tsv", sep="\t")

print("gpt4:")
df_gpt4_stories_questions = add_questions(df_gpt4_stories, "./outputs/294-doctrines-gpt4")
df_gpt4_stories_questions.to_csv("./data/294-doctrines/gpt4_story_question_294.tsv", sep="\t")

llama2:


100%|██████████| 294/294 [00:00<00:00, 23488.10it/s]


gpt3.5:


100%|██████████| 294/294 [00:00<00:00, 18469.91it/s]


gpt4:


100%|██████████| 294/294 [00:00<00:00, 26920.61it/s]


## 3. split 101 and 20 concepts from 294 for evaluation
- prepare stories which are conditioned to generate questions

In [124]:
# load 20 and 101 concepts

df_20 = pd.read_csv("./data/20-doctrines/legal_doctrines_20.tsv", sep="\t")[['concept', 'intro_text']]
print(df_20.shape)
df_101 = pd.read_csv("./data/101-doctrines/legal_doctrines_101.tsv", sep="\t")[['concept', 'intro_text']]
print(df_101.shape)

(20, 2)
(101, 2)


In [126]:
# LLaMA 2
model_name = "llama2"
print("Model Name:", model_name)

df_tmp = df_llama2_stories_questions.merge(df_20, how="inner", on=["concept", 'intro_text'])
print(df_tmp.shape)
df_tmp.to_csv("./data/20-doctrines/{}_story_question_20.tsv".format(model_name), sep="\t")

df_tmp = df_llama2_stories_questions.merge(df_101, how="inner", on=["concept", 'intro_text'])
print(df_tmp.shape)
df_tmp.to_csv("./data/101-doctrines/{}_story_question_101.tsv".format(model_name), sep="\t")


Model Name: llama2
(20, 6)
(101, 6)


In [127]:
# GPT 3.5
model_name = "gpt3.5"
print("Model Name:", model_name)

df_tmp = df_gpt35_stories_questions.merge(df_20, how="inner", on=["concept", 'intro_text'])
print(df_tmp.shape)
df_tmp.to_csv("./data/20-doctrines/{}_story_question_20.tsv".format(model_name), sep="\t")

df_tmp = df_gpt35_stories_questions.merge(df_101, how="inner", on=["concept", 'intro_text'])
print(df_tmp.shape)
df_tmp.to_csv("./data/101-doctrines/{}_story_question_101.tsv".format(model_name), sep="\t")


Model Name: gpt3.5
(20, 6)
(101, 6)


In [128]:
# GPT-4
model_name = "gpt4"
print("Model Name:", model_name)

df_tmp = df_gpt4_stories_questions.merge(df_20, how="inner", on=["concept", 'intro_text'])
print(df_tmp.shape)
df_tmp.to_csv("./data/20-doctrines/{}_story_question_20.tsv".format(model_name), sep="\t")

df_tmp = df_gpt4_stories_questions.merge(df_101, how="inner", on=["concept", 'intro_text'])
print(df_tmp.shape)
df_tmp.to_csv("./data/101-doctrines/{}_story_question_101.tsv".format(model_name), sep="\t")


Model Name: gpt4
(20, 6)
(101, 6)
